<a href="https://colab.research.google.com/github/All4Nothing/hg-mldl/blob/main/7-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 심층 신경망

In [4]:
from tensorflow import keras 
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [5]:
from sklearn.model_selection import train_test_split
train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28*28)
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)

입력층과 출력층 사이에 있는 모든 층을  **은닉층 hidden layer**라고 부른다

은닉층에는 활성화 함수가 적용된다.  
활성화 함수는 신경망 층의 선형 방정식의 계산 값에 적용하는 함수이다.  
출력층에 적용하는 활성화 함수는 종류가 제한되어 있다.(이진 분류의 경우 `시그모이드` 함수, 다중 분류의 경우 `소프트맥스` 함수)  
은닉층의 활성화 함수는 비교적 자유롭다. 대표적으로 `시그모이드` 함수와 `렐루 ReLU` 함수 등을 사용한다

분류 문제는 클래스에 대한 확률을 출력하기 위해 활성화 함수를 사용한다.  
회귀 출력은 임의의 어떤 숫자이므로 활성화 함수를 적용할 필요가 없다. 즉 출력층의 선형 방정식의 계산을 그대로 출력한다. Dense 층의 activation 매개변수에 아무런 값을 지정하지 않는다.

In [7]:
dense1 = keras.layers.Dense(100, activation = 'sigmoid', input_shape=(784,)) # 케라스에서 신경망의 첫 번째 층은 input_shape 매개변수로 입력의 크기를 꼭 지정해주어야 한다.
dense2 = keras.layers.Dense(10, activation='softmax')

`Dense 클래스`  
`dense1` : 은닉층, 100개의 뉴런을 가진 밀집층. 은닉층의 뉴런 개수를 정하는데는 특별한 기준이 없다. 출력층의 뉴런보다는 많게 만들어야 한다.  
`dense2` : 출력층, 10개의 클래스를 분류하므로 10개의 뉴런.

## 심층 신경망 만들기

`dense1`과 `dense2` 객체를 `Sequential 클래스에 추가하여 **심층 신경망 deep neural network, DNN`을 만든다.

가장 처음 등장하는 은닉층에서 마지막 출력층의 순서로 리스트를 나열하여 전달한다

In [8]:
model = keras.Sequential([dense1, dense2])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


맨 첫 줄에 모델 이름  
층마다 층 이름, 클래스, 출력 크기, 모델 파라미터 개수가 출력된다. 층을 만들 때 name 매개변수로 이름을 지정할 수 있고, 지정하지 않으면 자동으로 `dense`라고 이름을 붙인다

출력 크기 `(None, 100)`.   
1. 첫번째 차원은 샘플의 개수를 나타낸다. 케라스 모델의 `fit()` 메서드에 훈련 데이터를 주입하면 미니배치 경사 하강법을 사용하여 단계를 수행한다.  
케라스의 기본 미니배치 크기는 32개이다. `fit()` 메서드에서 `batch_size` 매개변수로 바꿀 수 있다.  
샘플 개수를 고정하지 않고 어떤 배치 크기에도 유연하게 대응할 수 있도록 None으로 설정  
신경망 층에 입력되거나 출력되는 배열의 첫번째 차원을 배치 차원이라고 부른다  
2. 은닉층의 뉴런 개수를 100개로 두었으니 100개의 출력이 나온다. 즉 샘플마다 784개의 픽셀값이 은닉층을 통과하면서 100개의 특성으로 압축되었다.  

모델 파라미터 개수 = 입력 x 뉴런수 + 뉴런수 x 1  
784개 x 100개 + 100개 = 78,500개  
100개 x 10개 + 10개 = 1,010개

훈련되지 않은 파라미터(Non-trainable params) : 경사 하강법으로 훈련되지 않는 파라미터를 가진 층의 파라미터 개수

## 층을 추가하는 다른 방법

1. Dense 클래스의 객체를 만들어 Sequential 클래스에 전달

2. Sequential 클래스의 생성자 안에서 바로 Dense 클래스의 객체를 만드는 경우

In [10]:
model = keras.Sequential([
  keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'),
  keras.layers.Dense(10, activation='softmax', name='output')                        
], name='패션 MNIST 모델')

In [11]:
model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


3. `add()`

In [14]:
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax'))

In [15]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 100)               78500     
                                                                 
 dense_3 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


훈련

In [17]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5623 - accuracy: 0.8077
Epoch 2/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.4088 - accuracy: 0.8527
Epoch 3/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3737 - accuracy: 0.8654
Epoch 4/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3513 - accuracy: 0.8725
Epoch 5/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3350 - accuracy: 0.8780


## 렐루 함수

시그모이드 함수는 오른쪽과 왼쪽 끝으로 갈수록 그래프가 누워있기 때문에 올바른 출력을 만드는데 신속하게 대응하지 못한다.  
특히 층이 많은 심층 신경망일수록 그 효과가 누적되어 학습을 더 어렵게 만든다

**렐루 ReLU**함수는 입력이 양수일 경우 활성화 함수가 없는 것처럼 그냥 입력을 통과시키고 음수일 경우에는 0으로 만든다.  
![ReLU](https://t1.daumcdn.net/cfile/tistory/99F8D13359F5F58F32)
`max(0,z)`: z가 0보다 크면 z를 출력하고 z가 0보다 작으면 0을 출력한다.  
렐루 함수는 특히 이미지 처리에서 좋은 성능을 낸다.  
`Flatten`층은 이미지를 1차원 배열로 펼쳐서 입력하지 않아도, 배치 차원을 제외하고 나머지 입력 차원을 모두 일렬로 펼치는 역할만 한다.  
`Flatten`층은 입력층 바로 뒤에 추가한다.

In [20]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [21]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_5 (Dense)             (None, 100)               78500     
                                                                 
 dense_6 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


`Flatten`층을 신경망 모델에 추가하면 입력값의 차원을 짐작할 수 있다. 784개의 입력이 첫번째 은닉층에 전달된다는 것을 알 수 있다.  
케라스 API는 입력 데이터에 대한 전처리 과정을 될 수 있으면 모델에 포함한다.

In [22]:
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()
train_scaled = train_input / 255.0
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)

In [23]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 7s 3ms/step - loss: 0.5326 - accuracy: 0.8118
Epoch 2/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3960 - accuracy: 0.8572
Epoch 3/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3587 - accuracy: 0.8709
Epoch 4/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3360 - accuracy: 0.8801
Epoch 5/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3204 - accuracy: 0.8849


In [24]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 2ms/step - loss: 0.3661 - accuracy: 0.8766


[0.3661092519760132, 0.8765833377838135]

## 옵티마이저

**옵티마이저 optimizer** 다양한 종류의 경사 하강법 알고리즘

확률적 경사 하강법 `SGD`. 이름이 SGD이지만 1개의 샘플을 봅아서 훈련하지 않고 기본적으로 미니배치를 사용한다.

In [25]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics='accuracy')

In [26]:
# 동일한 코드
sgd = keras.optimizers.SGD()
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics='accuracy')

In [27]:
sgd = keras.optimizers.SGD(learning_rate = 0.1) # SGD 클래스의 학습률 기본값이 0.01일 때 이를 바꾸고 싶을때 사용

![옵티마이저](https://thebook.io/img/080324/132.jpg)

`SGD`클래스의 momentum 매개변수의 기본값은 0이다. 이를 0보다 큰 값으로 지정하면 마치 이전의 그레이디언트를 가속도처럼 사용하는 **모멘텀 최적화 momentum optimization**을 사용한다. 보통 `momentum`매개변수는 0.9이상을 지정한다.

`SGD`클래스의 `nesterov` 매개변수를 기본값 `False`에서 `True`로 바꾸면 **네스테로프 모멘텀 최적화 nesterov momentum optimization(또는 네스테로프 가속 경사)**를 사용한다.

In [28]:
sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True)

네스테로프 모멘텀은 모멘텀 최적화를 2번 반복하여 구현한다. 대부분의 경우 네스테로프 모멘텀 최적화가 기본 확률적 경사 하강법보다 더 나은 성능을 제공한다

모델이 최적점에 가까이 갈수록 학습률을 낮출 수 있다. 이렇게 하면 안정적으로 최적점에 수렴할 가능성이 높다. 이런 학습률을 **적응적 학습률 adaptive learning rate**이라고 한다. 

적응적 학습률을 사용하는 대표적인 옵티마이저는 Adagrad와 RMSprop이다. 각각 `compile()`메서드의 `optimizer`매개변수에 `adagrad`와 `rmsprop`로 지정할 수 있다. 

`optimizer`매개변수의 기본값은 `rmsprop`이다

In [30]:
adagrad = keras.optimizers.Adagrad()
model.compile(optimizer=adagrad, loss='sparse_categorical_crossentropy', metrics='accuracy')

In [31]:
rmsprop = keras.optimizers.RMSprop()
model.compile(optimizer=rmsprop, loss='sparse_categorical_crossentropy', metrics='accuracy')

모멘텀 최적화와 RMSprop의 장점을 접목한 것이 `Adam`이다.

적응적 학습률을 사용하는 `RMSprop`, `Adagrad`, `Adam` 모두 `learning_rate` 매개변수의 기본값으로 모두 0.001을 사용한다

In [34]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [35]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 6s 3ms/step - loss: 0.5226 - accuracy: 0.8178
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3928 - accuracy: 0.8589
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3520 - accuracy: 0.8717
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3249 - accuracy: 0.8813
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3096 - accuracy: 0.8861


In [36]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 3ms/step - loss: 0.3395 - accuracy: 0.8768


[0.3395117521286011, 0.8768333196640015]